In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams['lines.linewidth'] = 0.2

import numpy as np
import pandas as pd

import experiments

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


In [23]:
class PowerTraceDataset(Dataset):
    def __init__(self, pickle_file, trace_size, transform=None):
        self.df = pd.read_pickle(pickle_file)
        self.trace_size = trace_size
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        plain, key, cipher, trace = self.df.iloc[index, :4]

        if trace.size < self.trace_size:
            trace = np.pad(trace, (0, self.trace_size), constant_values=0)
        else:
            trace = trace[:self.trace_size]

        trace = np.expand_dims(trace, 0)

        sample = {
            'plain': plain,
            'key': key,
            'cipher': cipher,
            'trace': trace
        }

        return sample



In [24]:
ds = PowerTraceDataset('./data/DPO4032_10_211102-181109.pkl', trace_size=35000)

In [25]:
dl = DataLoader(ds, batch_size=4, shuffle=True)

In [26]:
feat = next(iter(dl))

In [28]:
tr = feat['trace']
tr.shape # (4, 1, 35000) batch, channel, time

torch.Size([4, 1, 35000])